# Загрузка библиотек

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import requests
import zipfile as zf
import os
import shutil


# Загрузка данных

In [2]:
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

In [ ]:
req = requests.get(url, stream=True)

with open('kion.zip', 'wb') as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(
        desc='kion dataset download',
        total=total_size_in_bytes,
        unit='iB',
        unit_scale=True,
    )
    for chunk in req.iter_content(chunk_size=2**20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

In [7]:
with zf.ZipFile('kion.zip', 'r') as files:
    files.extractall('../datasets/kion')

source_dir = '../datasets/kion/data_original'
target_dir = '../datasets/kion'

for filename in os.listdir(source_dir):
    shutil.move(
        os.path.join(source_dir, filename), os.path.join(target_dir, filename)
    )


os.rmdir(source_dir)
os.remove('kion.zip')


In [8]:
users = pd.read_csv('../datasets/kion/users.csv')

В данном файле содержится информация о пользователях:

**user_id** - ID пользователя

**age** - возрастная группа пользователя, строка вида "M_N"
* 18_24 - от 18 до 24 лет включительно
* 25_34 - от 25 до 34 лет включительно
* 35_44 - от 35 до 44 лет включительно
* 45_54 - от 45 до 54 лет включительно
* 55_64 - от 55 до 64 лет включительно
* 65_inf - от 65 и старше

**sex** - пол пользователя
* М - мужчина
* Ж - женщина

**income** - доход пользователя, строка вида "M_N"
* income_0_20
* income_20_40
* income_40_60
* income_60_90  
* income_90_150
* income_150_inf

**kids_flg** - флаг "наличие ребенка"

In [9]:
items = pd.read_csv('../datasets/kion/items.csv')

**item_id** - ID контента \
**content_type**- Тип контента (фильм, сериал) \
**title** - Название на русском \
**title_orig** - Название оригинальное \
**genres** - Жанры из источника (онлайн-кинотеатры) \
**countries** - страны \
**for_kids** - флаг "контент для детей" \
**age_rating** - возрастной рейтинг \
**studios** - студии \
**directors** - директора \
**actors** - актеры \
**keywords** - ключевые слова \
**description** - описание

In [10]:
interactions = pd.read_csv('../datasets/kion/interactions.csv')

**user_id** - ID пользователя \
**item_id** - ID контента \
**last_watch_dt** - Дата последнего просмотра \
**total_dur** - Общая продолжительность всех просмотров данного контента в секундах \
**watched_pct** - сколько % фильма просмотрено

# Визуальный анализа данных

## Users/Пользователи

In [ ]:
users

In [ ]:
users.info()

In [ ]:
users.isnull().sum()

Есть пропуски, которые в дальнейшем надо будет обработать.

In [ ]:
users.loc[users.age.isna(), 'age'] = 'age_unknown'
users.loc[users.income.isna(), 'income'] = 'income_unknown'
users.loc[users.sex.isna(), 'sex'] = 'sex_unknown'

users.isnull().sum()

Пропусков больше нет. Также изменим некоторые типы данных (данные будут меньше потреблять оперативной памяти)

In [ ]:
users['sex'] = users['sex'].astype('category')
users['age'] = users['age'].astype('category')
users['income'] = users['income'].astype('category')
users['kids_flg'] = users['kids_flg'].astype('bool')

users.info()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, sharey=True, figsize=(16, 4))

# кол-во пользователей по возрастным группам
users.age.value_counts().plot.bar(ax=axes[0])

# кол-во пользователей по уровню дохода
users.income.value_counts().plot.bar(ax=axes[1])

# кол-во пользователей по полу
users.sex.value_counts().plot.bar(ax=axes[2])

# кол-во пользователей без детей и с детьми
users.kids_flg.value_counts().plot.bar(ax=axes[3])

plt.show()

Самая многочисленная аудитория в возрастной категории от 25 до 34 лет, самая малочисленная - от 65 лет и старше.

Самая многочисленная аудитория в категории доход от 20 до 40, самая малочисленная - от 150 и более.

Распределение по полу пользователей практически одинаково.

Бездетных пользователей примерно в 2 раза больше чем пользователей с детьми.

Проверим, что id пользователей не задублировались.

In [ ]:
users.duplicated(subset=['user_id']).sum()

## Items/Контент

In [ ]:
items.head()

In [ ]:
items.info()

In [ ]:
items.nunique()

### content_type

In [ ]:
items.content_type.value_counts().plot.bar(figsize=(2, 2))

In [ ]:
items.content_type.isna().sum()

Основной контент видеосервиса - фильмы. Пропусков нет.

In [23]:
items.content_type = items.content_type.astype('category')

### title,	title_orig

In [ ]:
# пропуски?
items.title.isna().sum()

In [ ]:
# пропуски?
items.title_orig.isna().sum()

Не у всех объектов заполнен признак title_orig, заполним их

In [ ]:
items.loc[items.title_orig.isna(), 'title_orig'] = 'None_title_orig'
items.title_orig.isna().sum()

Заменим типы данных

In [27]:
items.title = items.title.astype('category')
items.title_orig = items.title_orig.astype('category')

Так как одно и тоже слово может быть написано текстом разного регистра приведем весь текст к нижнему.

In [28]:
items.title = items.title.str.lower()

### release_year

In [ ]:
# кол-во контента по годам выпуска
items.release_year.value_counts().sort_index().plot.bar(figsize=(18, 4))

С каждым годом видеоконтента снимается все больше.

In [ ]:
# есть ли пропуски?
items.release_year.isna().sum()

In [ ]:
# посмотрим на эти объекты
items[items.release_year.isna()].head()

In [ ]:
# заменим
items.loc[items.release_year.isna(), 'release_year'] = 'release_year_unknown'
items.release_year.isna().sum()

Добавляем объединия по десятилетиям (1990-1999, 2000-2009, 2010-2019 ит.д)

In [33]:
items['release_year_int'] = pd.to_numeric(items['release_year'], errors='coerce')

items['release_decade'] = items['release_year_int'] // 10 * 10
items['release_decade'] = items['release_decade'].astype(str) + 's'

items['release_decade'] = items['release_decade'].replace(
    'nans', 'release_year_unknown'
)


In [ ]:
items.release_decade.value_counts().sort_index().plot.bar(figsize=(18, 4))

### genres

In [ ]:
items.genres.value_counts()

In [ ]:
# пропуски?
items.genres.isna().sum()

In [37]:
# приведем текст к нижнему регистру
items.genres = items.genres.str.lower()

In [38]:
items.genres = items.genres.astype('category')

### countries

In [ ]:
# Top-10 стран по количеству контента в базе
items.countries.value_counts().head(10).plot.bar()

Самые крупные страны-производители фильмов в данном видеосервисе - это Россия и США.

In [40]:
# приведем к нижнему регистру
items.countries = items.countries.str.lower()

In [ ]:
# пропуски?
items.countries.isna().sum()

In [ ]:
items.loc[items.countries.isna(), 'countries'] = 'countries_unknown'
items.countries.isna().sum()

In [43]:
items.countries = items.countries.astype('category')

### for_kids

In [ ]:
# пропуски
items.for_kids.isna().sum()

Посмотрим какие возрастные категории обозначены флагом for_kids

In [ ]:
items[items.for_kids == 1][['for_kids', 'age_rating']].value_counts()

In [ ]:
items[items.for_kids == 0][['for_kids', 'age_rating']].value_counts()

Было бы логично, что признак установлен для возрастной категории 0+ и 6+. Но есть случаи когда и на 0+ и на 6+ стоит значение 0. \
Очень много пропусков, не будем использовать этот признак.

### age_rating

In [ ]:
# Кол-во фильмов по рейтингам
items.age_rating.value_counts().plot.bar()

Самый большой выбор контента в категории 16+.

In [ ]:
# пропуски?
items.age_rating.isna().sum()

Всего два пропуска, посмотрим их.

In [ ]:
items[items.age_rating.isna()]

Это детские мультики 0+

In [ ]:
items.loc[items.age_rating.isna(), 'age_rating'] = 0
items.age_rating.isna().sum()

Данный признак является числовым, переделаем его в категориальный

In [51]:
items.age_rating = items.age_rating.astype('category')

### studios

In [ ]:
items.studios.value_counts().plot.bar(figsize=(18, 4))

Топ-3 - это студии HBO, Ленфильм и Sony Pictures

In [ ]:
# пропуски
items.studios.isna().sum()

In [ ]:
items.loc[items.studios.isna(), 'studios'] = 'studios_unknown'
items.studios.isna().sum()

In [55]:
# приведем к нижнему регистру
items.studios = items.studios.str.lower()

In [56]:
items.studios = items.studios.astype('category')

### directors

In [ ]:
items.directors.value_counts()

In [ ]:
items.directors.isna().sum()

In [ ]:
items.directors.fillna('directors_unknown', inplace=True)
items.directors.isna().sum()

In [60]:
# приведем к нижнему регистру
items.directors = items.directors.str.lower()

In [61]:
items.directors = items.directors.astype('category')

### actors

In [ ]:
items.actors.value_counts()

In [ ]:
items.actors.isna().sum()

In [ ]:
items.actors.fillna('actors_unknown', inplace=True)
items.actors.isna().sum()

In [65]:
# приведем к нижнему регистру
items.actors = items.actors.str.lower()

In [66]:
items.actors = items.actors.astype('category')

### description

In [ ]:
items.description

In [ ]:
items.description.isna().sum()

In [ ]:
items.description.fillna('None_description', inplace=True)
items.description.isna().sum()

### keywords

In [ ]:
items.keywords

In [ ]:
items.keywords.isna().sum()

Тут можно попробовать еще вариант - подобрать ключевые слова из description. Пока заполним None

In [ ]:
items.keywords.fillna('None_keywords', inplace=True)
items.keywords.isna().sum()

### Повторы

Посмотрим, есть ли дубликаты.

In [ ]:
# по id фильма
items[items.duplicated(subset=['item_id'], keep=False)]

In [ ]:
# по нескольким признакам
items[
    items.duplicated(
        subset=['content_type', 'countries', 'title', 'directors'], keep=False
    )
]

In [ ]:
# удалим обнаруженный дубликат
items.drop_duplicates(
    subset=['content_type', 'countries', 'title', 'directors'], inplace=True
)
items[
    items.duplicated(
        subset=['content_type', 'countries', 'title', 'directors'], keep=False
    )
]

### Итог

In [ ]:
items.info()

Пропущенные значения заполнили определнными значениями, при необходимости их можно будет заменить.

## Interactions/Взаимодействия

In [ ]:
interactions

In [ ]:
interactions.isnull().sum()

In [ ]:
sns.histplot(interactions.watched_pct)

Большая часть досматривает видео до конца. Также очень много тех, кто перестает смотреть сразу же. И, конечно, есть варианты между ними.

In [ ]:
# пропуски заменим нулем
interactions.watched_pct.fillna(0, inplace=True)
interactions.watched_pct.isna().sum()

Как вариант еще. Пропущенные значения watched_pct можно посчиать, так как известно время просмотра, только для этого нужно откуда-то спарсить общую продолжительность фильма.

### Повторы

In [ ]:
# проверка на дубли
interactions.duplicated(subset=['user_id', 'item_id'], keep=False).sum()

# Еще немного статистики

В users: пользователи + фичи \
В items: объекты + фичи \
В interactions: пользователи + объекты + взаимодействия 

In [82]:
# пользователи с фичами и взаимодействиями - это пересечение users & interactions
users_features_interactions = list(
    set(users.user_id) & set(interactions.user_id.unique())
)

# пользователи только с фичами - это users минус interactions
users_only_features = list(
    set(users.user_id) - set(interactions.user_id.unique())
)

# пользователи только с действиями - это interactions минус users
users_only_interactions = list(
    set(interactions.user_id.unique()) - set(users.user_id)
)

# всего пользователей
users_total = (
    users_features_interactions + users_only_features + users_only_interactions
)

In [ ]:
print(f'Кол-во пользователей всего - {len(users_total)}')
print(f'Кол-во пользователей только c фичами - {len(users_only_features)}')
print(
    f'Кол-во пользователей только c взаимодействиями - {len(users_only_interactions)}'
)
print(
    f'Кол-во пользователей c взаимодействиями и фичами - {len(users_features_interactions)}'
)

In [84]:
# объекты с фичами и взаимодействиями - это пересечение  items & interactions
items_features_interactions = list(
    set(items.item_id) & set(interactions.item_id.unique())
)

# объекты только с фичами - это items минус interactions
items_only_features = list(
    set(items.item_id) - set(interactions.item_id.unique())
)

# объекты только с действиями - это interactions минус items
items_only_interactions = list(
    set(interactions.item_id.unique()) - set(items.item_id)
)

# всего пользователей
items_total = (
    items_features_interactions + items_only_features + items_only_interactions
)

In [ ]:
print(f'Кол-во объектов всего - {len(items_total)}')
print(f'Кол-во объектов только c фичами - {len(items_only_features)}')
print(
    f'Кол-во объектов только c взаимодействиями - {len(items_only_interactions)}'
)
print(
    f'Кол-во объектов c взаимодействиями и фичами - {len(items_features_interactions)}'
)

# Сохранение

Сохраним подготовленные данные

In [86]:
users.to_csv(r'../datasets/users_processed.csv', index=False)
items.drop(columns=['for_kids']).to_csv(
    r'../datasets/items_processed.csv',
    index=False,
)
interactions.to_csv(r'../datasets/interactions_processed.csv', index=False)